In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from depsurf import CollisionType, InlineType, Versions
from depsurf.output import (
    get_legend_handles_labels,
    label_multiline_text,
    load_df,
    plot_yticks,
    save_fig,
)


def plot_subfig(ax: plt.Axes, df: pd.DataFrame, group: Versions, ylim, lines_fn):
    xs = np.arange(len(df))
    bottom = np.zeros(len(df))
    for col in df.columns:
        ax.bar(xs, df[col], bottom=bottom, label=col, color=col.color)
        bottom += df[col]

    ax.tick_params(axis="both", labelsize=9)
    ax.set_xticks(xs, [group.str_fn(v) for v in df.index], rotation=0)
    ax.set_xlabel(group.caption)
    ax.set_xlim(-0.5, len(df) - 0.5)
    ax.set_ylim(0, ylim)

    if lines_fn is None:
        return

    with plt.rc_context({"font.family": "monospace"}):
        for i, (index, row) in enumerate(df.iterrows()):
            lines, colors = zip(*lines_fn(row))
            label_multiline_text(ax, i, ylim, lines, colors=colors, fontsize=9)

groups = [Versions.REGULAR, Versions.ARCH, Versions.FLAVOR]
def plot_fig(df: pd.DataFrame, ylabel, lines_fn=None, ylim_mult=1.1):
    fig, axs = plt.subplots(
        1,
        3,
        figsize=(12, 2.25),
        sharey=True,
        width_ratios=Versions.num_versions(groups),
        gridspec_kw={"wspace": 0.05, "hspace": 0.075},
    )

    ylim = df.sum(axis=1).max() * ylim_mult
    for ax, group in zip(axs, groups):
        plot_subfig(ax, df.loc[group], group, ylim=ylim, lines_fn=lines_fn)

    plot_yticks(axs[0])
    axs[0].set_ylabel(ylabel)

    handles, labels = get_legend_handles_labels(fig)
    fig.legend(handles, labels, loc="upper center", ncol=5, bbox_to_anchor=(0.5, 1.02))
    return fig


fig = plot_fig(
    load_df("inline"),
    lines_fn=lambda row: [(f"{row[InlineType.PARTIAL]/1000:.1f}k", "darkred")],
    ylabel="Number of Functions",
    ylim_mult=1.1,
)
save_fig(fig, "inline")


fig = plot_fig(
    load_df("dup")[[CollisionType.INCLUDE, CollisionType.STATIC, CollisionType.MIXED]],
    lines_fn=lambda row: [
        (f"{row[CollisionType.MIXED]}", "darkgreen"),
        (f"{row[CollisionType.STATIC]}", "darkred"),
    ],
    ylabel="Number of Cases",
    ylim_mult=1.2,
)
save_fig(fig, "dup")

[       pd.py:50 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/inline.pkl
[      mpl.py:76 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/inline.pdf
[       pd.py:50 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/dup.pkl
[      mpl.py:76 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/dup.pdf
